# Going Deeper_03 텍스트의 분포로 벡터화 하기   


Word2Vec의 대중화 이전에, 텍스트의 분포를 활용하여 텍스트를 벡터화하는 아이디어를 들여다보자.   

1. 단어 빈도를 이용한 벡터화   
(1) Bag of Words   
(2) Bag of Words 구현해보기   
(3) DTM과 코사인 유사도   
(4) DTM의 구현과 한계점   
(5) TF-IDF   
(6) TF-IDF 구현하기   
2. LSA와 LDA   
(1) LSA   
(2) LSA 실습   
(3) LDA   
(4) LDA 실습   
3. 텍스트 분포를 이용한 비지도 학습 토크나이저   
(1) 형태소 분석기와 단어 미등록 문제   
(2) soynlp

텍스트를 벡터화하는 방법으로는 **(1) 통계와 머신러닝 활용**, **(2) 인공 신경망을 활용**하는 두가지 방법이 있다.   
이번엔 전자의 방법으로 가보자

## Bag of Words

BoW란, 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법이다. 텍스트를 전부 단어 단위로 토큰화 하고, 단어 사용 횟수를 카운트 한다.

In [5]:
doc1 = 'John likes to watch movies. Mary likes movies too.'
BoW1 = {"John":1, "likes":2, "to":1, "watch":1, "movies":2, "Mary":1, "too":1}

doc2 = 'Mary also likes to watch football games.'
BoW2 = {"Mary":1, "also":1, "likes":1, "to":1, "watch":1, "football":1, "games":1}

# 순서는 다르지만 둘 다 같다
BoW = {"too":1, "Mary":1, "movies":2, "John":1, "watch":1, "likes":2, "to":1}
BoW1 = {"John":1, "likes":2, "to":1, "watch":1, "movies":2, "Mary":1, "too":1}

doc3 = 'John likes to watch movies. Mary likes movies too. Mary also likes to watch football games.'
BoW3 = {"John":1, "likes":3, "to":2, "watch":2, "movies":2, "Mary":2, "too":1, "also":1, "football":1, "games":1}

어순이 달라지더라도 같은 문장으로 취급한다는 한계가 있다

## Keras Tokenizer로 Bag of Words 구현

In [6]:
# Keras Tokenizer로 Bag of Words 구현

from tensorflow.keras.preprocessing.text import Tokenizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence) # 텍스트를 리스트 형태로 단어장 생성 (중복 X)
bow = dict(tokenizer.word_counts) # 각 단어와 각 단어의 빈도를 bow에 저장

print("Bag of Words :", bow) # bow 출력
print('단어장(Vocabulary)의 크기 :', len(tokenizer.word_counts)) # 중복을 제거한 단어들의 개수

Bag of Words : {'john': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
단어장(Vocabulary)의 크기 : 10


## scikit-learn CountVectorizer로 구현

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

vector = CountVectorizer()
bow = vector.fit_transform(sentence).toarray()

print('Bag of Words : ', bow) # 코퍼스로부터 각 단어의 빈도수를 기록한다.
print('각 단어의 인덱스 :', vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.
print('단어장(Vocabulary)의 크기 :', len(vector.vocabulary_))

Bag of Words :  [[1 1 1 1 3 2 2 2 1 2]]
각 단어의 인덱스 : {'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}
단어장(Vocabulary)의 크기 : 10


scikit-learn의 CountVectorizer 빈도수만 나올 뿐이다.   
Keras의 토크나이저를 사용하는 것이 보통이다

# DTM(Document-Term Matrix)

DTM이란, **여러 문서의 Bag of Words를 하나의 행렬로 구현**한 것이다.   
즉, **각 문서에 등장한 단어의 빈도수를 하나의 행렬로 통합**한 것이다.

Doc 1: Intelligent applications creates intelligent business processes   
Doc 2: Bots are intelligent applications   
Doc 3: I do business intelligence   

![DTM](https://user-images.githubusercontent.com/59006548/145340168-a693bdfc-535d-48e6-b95d-97de94f53040.png)   
위 문장들로 만들어진 DTM   
   
row는 문서 벡터(document vector), column은 단어 벡터(word vector)   
문서 수가 많아지면 단어장이 커져서 희소벡터가 되어버린다.

In [8]:
# 코사인 유사도
# 문서1 : I like dog
# 문서2 : I like cat
# 문서3 : I like cat I like cat
# 위 문장의 DTM에서 코사인 유사도 계산
# https://wikidocs.net/24603

import numpy as np
from numpy import dot
from numpy.linalg import norm

doc1 = np.array([0,1,1,1]) # 문서1 벡터
doc2 = np.array([1,0,1,1]) # 문서2 벡터
doc3 = np.array([2,0,2,2]) # 문서3 벡터

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

![코사인유사도](https://wikidocs.net/images/page/24603/%EC%BD%94%EC%82%AC%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84.PNG)

![코사인 유사도](https://i.imgur.com/C6VnrI4.png)

In [9]:
print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

0.6666666666666667
0.6666666666666667
1.0000000000000002


`문서1`과 `문서2`의 코사인 유사도는 0.67, `문서1`과 `문서3`의 코사인 유사도도 0.67      
`문서2`와 `문서3`의 유사도는 1 (모든 단어의 빈도수가 동일하게 증가했기 때문)    
코사인 유사도는 벡터의 크기가 아니라 벡터의 방향(패턴)에 초첨을 둔다

In [10]:
# CountVectorizer로 DTM을 만들자

from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'John likes to watch movies',
    'Mary likes movies too',
    'Mary also likes to watch football games',    
]
vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도수를 기록.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[0 0 0 1 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 0 1 0]
 [1 1 1 0 1 1 0 1 0 1]]
{'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


### DTM의 한계

1. DTM의 문서 수와 단어 수가 늘어날 수록 벡터가 쓸데없이 커진다. (희소벡터, 차원의 저주)
2. 단어의 빈도에만 집중하는 방법이기에 한계가 있다. `the`가 많이 있다고 해서 유사한 문장 X   
- 그렇다면, 중요한 단어와 중요하지 않은 단어에 가중치를 따로 선별하는 방법은?

# TF-IDF

`TF-IDF`(Term Frequency-Inverse Document Frequency)는 모든 문서에서 자주 등장하는 단어는 중요도를 낮게 보고, 특정 문서에서만 자주 등장하는 단어는 중요도를 높게 본다. 마치 불용어를 제외하고 보듯이. -> IDF 항에서 이 역할을 수행   
하지만 이것이 DTM보다 성능이 항상 좋지는 않다.   
DTM을 만든 뒤 TF-IDF 가중치를 DTM에 적용   
사실 DTM 자체가 이미 TF (Term Frequency)   
![TF-IDF](https://img1.daumcdn.net/thumb/R800x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbKcggU%2FbtqCkQ2NEH1%2FAp9xO7HQSDzfKixMyuGNCk%2Fimg.png)   
tf 뒤에 곱해지는 log항이 IDF.   

전체 문서의 수가 5개라고 해봅시다. 그리고 단어 **'like'가 문서2에서 200번**, **문서 3에서 300번** 등장했다고 해봅시다. 다른 문서에서 단어 `'like'`는 등장하지 않았습니다. 이때, **단어 'like'의 IDF**는 몇일까요?
($$tf1=200,tf2=300, N=500, df=200 $$)

$$IDF = log(500/200) = ln(5/2) = 0.91629073187$$   
그러면 여기서 문서2와 문서3의 단어 `'like'`의 TF-IDF의 값은?   
$$문서2 TF-IDF = 200 * ln(5/2) = 183.258146375$$
$$문서3 TF-IDF = 300 * ln(5/2) = 274.887219562$$

위에꺼 계산 이거 맞아??

https://www.bloter.net/newsView/blt201609280001
희소벡터를 해결하기 위해 특이값분해를 통해 축소시킨다. 

In [21]:
from math import log
import pandas as pd

In [22]:
docs = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',  
]

In [23]:
# DTM의 열을 만들기 위해 문서 3개의 단어가 모두 들어간 통합 단어장을 만든다.

vocab = list({w for doc in docs for w in doc.split()}) # set comprehension으로 중복 단어 제거    
vocab.sort()
print('단어장의 크기 :', len(vocab))
print(vocab)

N = len(docs)
N

단어장의 크기 : 13
['James', 'John', 'Mary', 'TV', 'also', 'and', 'football', 'games', 'likes', 'movies', 'to', 'too', 'watch']


3

In [24]:
# TF-IDF 함수를 만드는 데, log 항에는 분모 1을 더해준다 (0 방지)

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1)) + 1 # log항에 또 1을 더해준다. 분자와 분모값이 같아져서 0이 되는 것을 방지

def tf_idf(t,d):
    return tf(t,d) * idf(t)

In [25]:
# TF 함수를 사용하여 DTM 생성

result = []
for i in range(N):
    result.append([]) # 빈 리스트 삽입
    d= docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        
        result[-1].append(tf(t,d)) # 빈 리스트에 tf 삽입


tf_  = pd.DataFrame(result, columns= vocab)
tf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0,1,1,0,0,1,0,0,2,2,2,1,1
1,1,0,0,1,0,0,0,0,1,0,1,0,1
2,0,0,1,0,1,0,1,1,1,0,1,0,1


In [26]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns=["IDF"])
idf_

,IDF
James,1.405465
John,1.405465
Mary,1.000000
TV,1.405465
also,1.405465
and,1.405465
football,1.405465
games,1.405465
likes,0.712318
movies,1.405465


In [27]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        
        result[-1].append(tf_idf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0.000000,1.405465,1.0,0.000000,0.000000,1.405465,0.000000,0.000000,1.424636,2.81093,1.424636,1.405465,0.712318
1,1.405465,0.000000,0.0,1.405465,0.000000,0.000000,0.000000,0.000000,0.712318,0.00000,0.712318,0.000000,0.712318
2,0.000000,0.000000,1.0,0.000000,1.405465,0.000000,1.405465,1.405465,0.712318,0.00000,0.712318,0.000000,0.712318


사이킷런에서 DTM을 만들 때 `CountVectorizer`를 사용했듯이,   
TF-IDF를 자동으로 계산하여 출력하는 TfidfVectorizer 사용 가능하다.   
여기에선 log항의 분자에도 1을 더해주며, TF-IDF 결과에 *L2 Norm*까지 추가로 수행한다

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',  
]

tfidfv = TfidfVectorizer().fit(corpus)
vocab = list(tfidfv.vocabulary_.keys()) # 단어장을 리스트로 저장
vocab.sort() # 단어장을 알파벳 순으로 정렬

# TF-IDF 행렬에 단어장을 데이터프레임의 열로 지정하여 데이터프레임 생성
tfidf_ = pd.DataFrame(tfidfv.transform(corpus).toarray(), columns = vocab)
tfidf_

,also,and,football,games,james,john,likes,mary,movies,to,too,tv,watch
0,0.000000,0.321556,0.000000,0.000000,0.000000,0.321556,0.379832,0.244551,0.643111,0.189916,0.321556,0.000000,0.189916
1,0.000000,0.000000,0.000000,0.000000,0.572929,0.000000,0.338381,0.000000,0.000000,0.338381,0.000000,0.572929,0.338381
2,0.464997,0.000000,0.464997,0.464997,0.000000,0.000000,0.274634,0.353642,0.000000,0.274634,0.000000,0.000000,0.274634


# LSA

LSA(Latent Semantic Analysis) 는 전체 코퍼스에서 문서 속 단어들 사이의 관계를 찾아내는 자연어 처리 정보 검색 기술   
LSA를 사용하면 단어와 단어 사이, 문서와 문서 사이, 단어와 문서 사이의 의미적 유사성 점수를 찾아낼 수 있다.

[고유값(eigenvalue), 고유벡터(eigenvector), 고유값 분해(eigen decomposition)](https://bkshin.tistory.com/entry/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-19-%ED%96%89%EB%A0%AC?category=1057680)   
[특잇값 분해](https://datascienceschool.net/02%20mathematics/03.04%20%ED%8A%B9%EC%9E%87%EA%B0%92%20%EB%B6%84%ED%95%B4.html)   
[Singular value decomposition의 목적](https://youtu.be/vxJ1MzfvL5w)

![SVD](https://user-images.githubusercontent.com/59006548/145714802-5df67202-5b3f-4b13-8ed7-0ed0c19d4747.png)   
LSA는 DTM이나 TF-IDF 행렬 등에 Truncated SVD를 수행   
Truncated SVD를 수행하면 행렬 Σ의 대각 원솟값 중에서 상윗값 t개만 남게 되며, U행렬과 V행렬의 t열까지만 남는다. 이로 인해 세 행렬에서 값(정보)의 손실이 일어나 기존의 행렬 A를 정확히 복구할 수는 없다.   
여기서 t는 하이퍼파라미터. t를 크게 잡으면 기존의 행렬 A로부터 다양한 의미를 가져갈 수 있지만, 노이즈를 제거하려면 t를 작게 잡아야 한다.   
여기서 얻은 $$U_k, V_k^T, S$$는 각각  '문서들과 관련된 의미들을 표현한 행렬', '단어들과 관련된 의미를 표현한 행렬' , '각 의미의 중요도를 표현한 행렬' 이라고 해석할 수 있다.   
Uk는 mXk의 크기를 가지는데 m은 문서 벡터로, 줄어들지 않는다.   
VkT는 kXn으로 저차원으로 축소되었다. k열은 전체 코퍼스로부터 얻어낸 kk개의 주요 주제(topic)라고 할 수 있다.

In [29]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [30]:
# nltk 데이터셋 다운
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /aiffel/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /aiffel/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [31]:
# 데이터 다운로드

import os

csv_filename = os.getenv('HOME')+'/aiffel/topic_modelling/data/abcnews-date-text.csv'

urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", 
                           filename=csv_filename)

('/aiffel/aiffel/topic_modelling/data/abcnews-date-text.csv',
 <http.client.HTTPMessage at 0x7fdc9420e750>)

In [32]:
data = pd.read_csv(csv_filename, error_bad_lines=False)
data.shape

(1082168, 2)

In [33]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [34]:
# 헤드라인만 가져오자
text = data[['headline_text']].copy()
text.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [35]:
text.nunique() # 중복을 제외하고 유일한 시퀀스를 가지는 샘플의 개수를 출력

headline_text    1054983
dtype: int64

In [36]:
text.drop_duplicates(inplace=True) # 중복 샘플 제거
text.reset_index(drop=True, inplace=True)
text.shape

(1054983, 1)

### 데이터 정제 및 정규화

In [37]:
# NLTK 토크나이저를 이용해서 토큰화
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

# 불용어 제거
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop_words)])

text.head()

,headline_text
0,"[aba, decides, community, broadcasting, licence]"
1,"[act, fire, witnesses, must, aware, defamation]"
2,"[g, calls, infrastructure, protection, summit]"
3,"[air, nz, staff, aust, strike, pay, rise]"
4,"[air, nz, strike, affect, australian, travellers]"


In [38]:
# 단어 정규화. 3인칭 단수 표현 -> 1인칭 변환, 과거형 동사 -> 현재형 동사 등을 수행한다.
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

# 길이가 1 ~ 2인 단어는 제거.
text = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 2])
print(text[:5])

0     [aba, decide, community, broadcast, licence]
1    [act, fire, witness, must, aware, defamation]
2       [call, infrastructure, protection, summit]
3            [air, staff, aust, strike, pay, rise]
4    [air, strike, affect, australian, travellers]
Name: headline_text, dtype: object
